In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import json

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
with open('config.json') as json_data_file:
            data = json.load(json_data_file)

In [14]:
data['description']

'This is a JSON configuration file for the CNN model tuning for CIFAR-10 Dataset.'

In [15]:
data

{'Activation': ['softmax',
  'elu',
  'selu',
  'softplus',
  'softsign',
  'relu',
  'tanh',
  'sigmoid',
  'hard_sigmoid',
  'exponential',
  'linear'],
 'compile': {'Decay': 1e-06,
  'Learning_Rate': 0.001,
  'Optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax'],
  'loss': ['categorical_crossentropy',
   'mean_squared_error',
   'mean_absolute_error',
   'mean_absolute_percentage_error',
   'mean_squared_logarithmic_error',
   'squared_hinge',
   'hinge',
   'categorical_hinge',
   'logcosh',
   'sparse_categorical_crossentropy',
   'binary_crossentropy',
   'kullback_leibler_divergence',
   'poisson',
   'cosine_proximity']},
 'description': 'This is a JSON configuration file for the CNN model tuning for CIFAR-10 Dataset.',
 'fit': {'Batch_Size': 32,
  'Data_Augmentation': True,
  'Num_Classes': 10,
  'Shuffle': False,
  'epochs': 50}}

In [16]:
batch_size = data['fit']['Batch_Size']
num_classes = data['fit']['Num_Classes']
epochs = data['fit']['epochs']
data_augmentation = data['fit']['Shuffle']
#num_predictions = 20
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'keras_cifar10_trained_model.h5'

In [17]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [18]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model1.add(Activation('relu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [20]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=data['compile']['Learning_Rate'], decay=data['compile']['Decay'])

In [1]:
# Let's train the model1 using RMSprop
model1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


NameError: name 'model1' is not defined

In [22]:
if not data_augmentation:
    print('Not using data augmentation.')
    model1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model1 on the batches generated by datagen.flow().
    history = model1.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 283s 6ms/step - loss: 1.8311 - acc: 0.3290 - val_loss: 1.5631 - val_acc: 0.4317
Epoch 2/50
50000/50000 [==============================] - 291s 6ms/step - loss: 1.5062 - acc: 0.4505 - val_loss: 1.3419 - val_acc: 0.5186
Epoch 3/50
50000/50000 [==============================] - 288s 6ms/step - loss: 1.3636 - acc: 0.5122 - val_loss: 1.2494 - val_acc: 0.5573
Epoch 4/50
50000/50000 [==============================] - 277s 6ms/step - loss: 1.2691 - acc: 0.5495 - val_loss: 1.1649 - val_acc: 0.5894
Epoch 5/50
50000/50000 [==============================] - 275s 5ms/step - loss: 1.1917 - acc: 0.5785 - val_loss: 1.1984 - val_acc: 0.5740
Epoch 6/50
50000/50000 [==============================] - 281s 6ms/step - loss: 1.1194 - acc: 0.6066 - val_loss: 1.0301 - val_acc: 0.6412
Epoch 7/50
30784/50000 [=================>............] - ETA: 1:44 - loss: 1.0675 - acc: 0.6

KeyboardInterrupt: 